In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from matplotlib import pyplot
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

In [2]:
DataFrame = pd.read_csv('../Data.csv')

In [3]:
QB = DataFrame[DataFrame['Position'] == 'QB']

In [4]:
QB = QB.drop(columns = ['Week','Position','Game','Unnamed: 0'])

In [5]:
QB = QB.drop(columns = ['Receptions', 'Receiving Yards','Receiving TDs'])

In [6]:
QB

,Name,Year,Fantasy Points,PassingYards,PassingTDs,Ints,Rush Attempts,Rush Yards,Fumbles,RushTDs
0,Lamar Jackson,2019,42.0,324.0,5.0,0,3,6,0,0
1,Dak Prescott,2019,41.0,405.0,4.0,0,4,12,0,0
2,Deshaun Watson,2019,36.0,268.0,3.0,1,4,40,0,1
3,Matthew Stafford,2019,34.0,385.0,3.0,0,3,22,1,0
4,Case Keenum,2019,33.0,380.0,3.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
3193,Jalen Hurts,2023,0.0,55.0,0.0,1,2,4,0,0
3194,A.J. McCarron,2023,0.0,20.0,0.0,0,0,0,0,0
3195,Clayton Tune,2023,0.0,0.0,0.0,0,1,1,0,0
3196,Bailey Zappe,2023,0.0,88.0,0.0,2,3,13,0,0


In [7]:
QB = QB.groupby(['Name','Year']).sum()

In [8]:
QB.reset_index(inplace=True)
QB.set_index('Year', inplace = True)
QB.index = pd.to_datetime(QB.index, format='%Y')

In [9]:
QB

,Name,Fantasy Points,PassingYards,PassingTDs,Ints,Rush Attempts,Rush Yards,Fumbles,RushTDs
Year,,,,,,,,,
2019-01-01,A.J. McCarron,16.0,225.0,0.0,1,5,39,0,1
2020-01-01,A.J. McCarron,0.0,20.0,0.0,0,0,0,0,0
2023-01-01,A.J. McCarron,0.0,19.0,0.0,0,0,0,0,0
2019-01-01,Aaron Rodgers,322.0,4002.0,26.0,4,46,183,4,1
2020-01-01,Aaron Rodgers,469.0,4299.0,48.0,5,38,149,2,3
...,...,...,...,...,...,...,...,...,...
2019-01-01,Will Grier,0.0,228.0,0.0,4,7,22,1,0
2023-01-01,Will Levis,115.0,1808.0,8.0,4,25,57,4,1
2021-01-01,Zach Wilson,163.0,2334.0,9.0,11,29,185,1,4


In [15]:
QB

,Name,Fantasy Points,PassingYards,PassingTDs,Ints,Rush Attempts,Rush Yards,Fumbles,RushTDs
Year,,,,,,,,,
2019-01-01,A.J. McCarron,16.0,225.0,0.0,1,5,39,0,1
2020-01-01,A.J. McCarron,0.0,20.0,0.0,0,0,0,0,0
2023-01-01,A.J. McCarron,0.0,19.0,0.0,0,0,0,0,0
2019-01-01,Aaron Rodgers,322.0,4002.0,26.0,4,46,183,4,1
2020-01-01,Aaron Rodgers,469.0,4299.0,48.0,5,38,149,2,3
...,...,...,...,...,...,...,...,...,...
2019-01-01,Will Grier,0.0,228.0,0.0,4,7,22,1,0
2023-01-01,Will Levis,115.0,1808.0,8.0,4,25,57,4,1
2021-01-01,Zach Wilson,163.0,2334.0,9.0,11,29,185,1,4


In [16]:
X = QB.drop(['Fantasy Points'], axis=1)
y = QB['Fantasy Points']


numeric_features = ['PassingTDs']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
 
    ]
)
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])


X_train_subset = X_train[X_train.index.year < 2022]
y_train_subset = y_train[y_train.index.year < 2022]
pipeline.fit(X_train_subset, y_train_subset)


X_test_subset = X_test[X_test.index.year == 2022]
y_test_subset = y_test[y_test.index.year == 2022]
y_pred = pipeline.predict(X_test_subset)

mse = mean_squared_error(y_test_subset, y_pred)
r2 = r2_score(y_test_subset, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 355.5670208997699
R-squared: 0.9728684769074696


In [17]:
X = QB.drop(['Fantasy Points'], axis=1)
y = QB['Fantasy Points']


numeric_features = ['PassingTDs']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
 
    ]
)
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

X_train_subset = X_train[X_train.index.year < 2023]
y_train_subset = y_train[y_train.index.year < 2023]
pipeline.fit(X_train_subset, y_train_subset)


X_test_subset = X_test[X_test.index.year == 2023]
y_test_subset = y_test[y_test.index.year == 2023]
y_pred = pipeline.predict(X_test_subset)

mse = mean_squared_error(y_test_subset, y_pred)
r2 = r2_score(y_test_subset, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 871.4454749846559
R-squared: 0.9337078797727423


In [18]:
features = ['PassingYards']
target_variable = 'PassingTDs'

In [19]:
predictions_2024 = {}

players = QB['Name'].unique()

for player in players:
    player_data = QB[QB['Name'] == player]
    time_series = player_data[features + [target_variable]]
    
    try:
        # Fit an ARIMA model
        model = ARIMA(time_series[target_variable], order=(1, 0, 1))  # Adjust the order as needed
        arima_result = model.fit()
        
        # Predict Fantasy Points for 2024
        start_date_2024 = datetime(2024, 1, 1)
        end_date_2024 = datetime(2024, 12, 31)
        y_pred_2024 = arima_result.predict(start=start_date_2024, end=end_date_2024, dynamic=False)
        
        # Store predictions in the dictionary
        predictions_2024[player] = y_pred_2024.values[-1]  # Only store the prediction for 2024
        
        
        
    except Exception as e:
        continue



In [20]:
predictions_2024

{'Aaron Rodgers': 22.91761937051345,
 'Andy Dalton': 3.5997760511667565,
 'Baker Mayfield': 19.73161521569008,
 'Ben Roethlisberger': 22.146337721531424,
 'Blaine Gabbert': 0.009918797706637617,
 'Brandon Allen': 2.2816922424558412,
 'Brett Rypien': 6.770807359224307e-08,
 'Brian Hoyer': 0.7518355078701652,
 'C.J. Beathard': 1.2041686926261115,
 'Cam Newton': 4.84842176413109,
 'Carson Wentz': 16.499309295135898,
 'Case Keenum': 1.8390204508989159,
 'Chad Henne': 0.6163963240473662,
 'Chase Daniel': 1.4999861939329657,
 'Chris Streveler': 0.3082110404461854,
 'Colt McCoy': 1.3803728541628622,
 'Cooper Rush': 1.2288089463592662,
 'Dak Prescott': 33.69214908252157,
 'Daniel Jones': 8.126366843127887,
 'David Blough': 1.0063990469142068,
 'Davis Mills': 10.004672692474259,
 'Derek Carr': 24.84247192385446,
 'Gardner Minshew': 14.229105319965615,
 'Garrett Gilbert': 5.2199355765059074e-06,
 'Geno Smith': 14.410679907696155,
 'Jacoby Brissett': 5.3467971290978165,
 'Jalen Hurts': 18.3176011

In [21]:
Predictions24 = pd.DataFrame(list(predictions_2024.items()), columns=['Name', 'PassingTDs'])

In [22]:
Predictions24

,Name,PassingTDs
0,Aaron Rodgers,22.917619
1,Andy Dalton,3.599776
2,Baker Mayfield,19.731615
3,Ben Roethlisberger,22.146338
4,Blaine Gabbert,0.009919
...,...,...
61,Trevor Lawrence,21.490243
62,Tua Tagovailoa,24.036580
63,Tyler Huntley,2.830355
64,Tyrod Taylor,3.200256


In [23]:
Predictions24['Year'] = 2024

In [24]:
Predictions24.reset_index(inplace=True)
Predictions24.set_index('Year', inplace = True)
Predictions24.index = pd.to_datetime(Predictions24.index, format='%Y')
Predictions24.reset_index(inplace=True)
Predictions24.drop(columns = ['index'])

,Year,Name,PassingTDs
0,2024-01-01,Aaron Rodgers,22.917619
1,2024-01-01,Andy Dalton,3.599776
2,2024-01-01,Baker Mayfield,19.731615
3,2024-01-01,Ben Roethlisberger,22.146338
4,2024-01-01,Blaine Gabbert,0.009919
...,...,...,...
61,2024-01-01,Trevor Lawrence,21.490243
62,2024-01-01,Tua Tagovailoa,24.036580
63,2024-01-01,Tyler Huntley,2.830355
64,2024-01-01,Tyrod Taylor,3.200256


In [25]:
QB

,Name,Fantasy Points,PassingYards,PassingTDs,Ints,Rush Attempts,Rush Yards,Fumbles,RushTDs
Year,,,,,,,,,
2019-01-01,A.J. McCarron,16.0,225.0,0.0,1,5,39,0,1
2020-01-01,A.J. McCarron,0.0,20.0,0.0,0,0,0,0,0
2023-01-01,A.J. McCarron,0.0,19.0,0.0,0,0,0,0,0
2019-01-01,Aaron Rodgers,322.0,4002.0,26.0,4,46,183,4,1
2020-01-01,Aaron Rodgers,469.0,4299.0,48.0,5,38,149,2,3
...,...,...,...,...,...,...,...,...,...
2019-01-01,Will Grier,0.0,228.0,0.0,4,7,22,1,0
2023-01-01,Will Levis,115.0,1808.0,8.0,4,25,57,4,1
2021-01-01,Zach Wilson,163.0,2334.0,9.0,11,29,185,1,4


In [26]:
PassingTDs = QB.drop(columns = ['PassingYards','Ints','Rush Attempts','Rush Yards','Fumbles','RushTDs'])

In [27]:
PassingTDs.reset_index(inplace=True)

In [28]:
PassingTDs

,Year,Name,Fantasy Points,PassingTDs
0,2019-01-01,A.J. McCarron,16.0,0.0
1,2020-01-01,A.J. McCarron,0.0,0.0
2,2023-01-01,A.J. McCarron,0.0,0.0
3,2019-01-01,Aaron Rodgers,322.0,26.0
4,2020-01-01,Aaron Rodgers,469.0,48.0
...,...,...,...,...
393,2019-01-01,Will Grier,0.0,0.0
394,2023-01-01,Will Levis,115.0,8.0
395,2021-01-01,Zach Wilson,163.0,9.0
396,2022-01-01,Zach Wilson,105.0,6.0


In [29]:
concatenated_df = pd.concat([PassingTDs, Predictions24])

In [30]:
concatenated_df

,Year,Name,Fantasy Points,PassingTDs,index
0,2019-01-01,A.J. McCarron,16.0,0.000000,NaN
1,2020-01-01,A.J. McCarron,0.0,0.000000,NaN
2,2023-01-01,A.J. McCarron,0.0,0.000000,NaN
3,2019-01-01,Aaron Rodgers,322.0,26.000000,NaN
4,2020-01-01,Aaron Rodgers,469.0,48.000000,NaN
...,...,...,...,...,...
61,2024-01-01,Trevor Lawrence,NaN,21.490243,61.0
62,2024-01-01,Tua Tagovailoa,NaN,24.036580,62.0
63,2024-01-01,Tyler Huntley,NaN,2.830355,63.0
64,2024-01-01,Tyrod Taylor,NaN,3.200256,64.0


In [31]:
PassingTDs = concatenated_df.sort_values(by=['Name','Year'], ascending=True)

In [32]:
PassingTDs

,Year,Name,Fantasy Points,PassingTDs,index
0,2019-01-01,A.J. McCarron,16.0,0.000000,NaN
1,2020-01-01,A.J. McCarron,0.0,0.000000,NaN
2,2023-01-01,A.J. McCarron,0.0,0.000000,NaN
3,2019-01-01,Aaron Rodgers,322.0,26.000000,NaN
4,2020-01-01,Aaron Rodgers,469.0,48.000000,NaN
...,...,...,...,...,...
394,2023-01-01,Will Levis,115.0,8.000000,NaN
395,2021-01-01,Zach Wilson,163.0,9.000000,NaN
396,2022-01-01,Zach Wilson,105.0,6.000000,NaN
397,2023-01-01,Zach Wilson,132.0,8.000000,NaN


In [33]:
PassingTDs = PassingTDs.drop(columns = ['index'])

In [34]:
PassingTDs

,Year,Name,Fantasy Points,PassingTDs
0,2019-01-01,A.J. McCarron,16.0,0.000000
1,2020-01-01,A.J. McCarron,0.0,0.000000
2,2023-01-01,A.J. McCarron,0.0,0.000000
3,2019-01-01,Aaron Rodgers,322.0,26.000000
4,2020-01-01,Aaron Rodgers,469.0,48.000000
...,...,...,...,...
394,2023-01-01,Will Levis,115.0,8.000000
395,2021-01-01,Zach Wilson,163.0,9.000000
396,2022-01-01,Zach Wilson,105.0,6.000000
397,2023-01-01,Zach Wilson,132.0,8.000000


In [35]:
X = QB.drop(['Fantasy Points'], axis=1)
y = QB['Fantasy Points']


numeric_features = ['PassingTDs']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
      
    ]
)


pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])


X_train_subset = X_train[X_train.index.year < 2023]
y_train_subset = y_train[y_train.index.year < 2023]
pipeline.fit(X_train_subset, y_train_subset)


X_test_subset = X_test[X_test.index.year == 2023]
y_test_subset = y_test[y_test.index.year == 2023]
y_pred = pipeline.predict(X_test_subset)


mse = mean_squared_error(y_test_subset, y_pred)
r2 = r2_score(y_test_subset, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 871.4454749846559
R-squared: 0.9337078797727423


In [36]:
latest_predictions = PassingTDs.groupby('Name')['PassingTDs'].last().reset_index()

# Preprocess the data for prediction
X_2024 = latest_predictions[['PassingTDs']]

# Make predictions for fantasy points in 2024
y_pred_2024 = pipeline.predict(X_2024)

# Combine player names and their corresponding predicted fantasy points
player_predictions = pd.DataFrame({
    'Player': latest_predictions['Name'],
    'Predicted_Fantasy_Points_2024': y_pred_2024.round(2)  # Round predictions to 2 decimal places
})

# Print the predicted fantasy points for 2024


In [37]:
player_predictions

,Player,Predicted_Fantasy_Points_2024
0,A.J. McCarron,11.74
1,Aaron Rodgers,277.53
2,Aidan O'Connell,150.91
3,Alex Smith,81.32
4,Alex Tanney,11.74
...,...,...
128,Tyrod Taylor,48.85
129,Tyson Bagent,46.53
130,Will Grier,11.74
131,Will Levis,104.52


In [38]:
DataFrame = pd.read_csv('../Data.csv')
QB2 = DataFrame[DataFrame['Position'] == 'QB']
QB2 = QB2.drop(columns = ['Receiving Yards','Receptions','Receiving TDs'])
QB2 = QB2.drop(columns = ['Week','Position','Game','Unnamed: 0'])
QB2 = QB2.groupby(['Name','Year']).sum()
QB2.reset_index(inplace=True)

In [39]:
QB2

,Name,Year,Fantasy Points,PassingYards,PassingTDs,Ints,Rush Attempts,Rush Yards,Fumbles,RushTDs
0,A.J. McCarron,2019,16.0,225.0,0.0,1,5,39,0,1
1,A.J. McCarron,2020,0.0,20.0,0.0,0,0,0,0,0
2,A.J. McCarron,2023,0.0,19.0,0.0,0,0,0,0,0
3,Aaron Rodgers,2019,322.0,4002.0,26.0,4,46,183,4,1
4,Aaron Rodgers,2020,469.0,4299.0,48.0,5,38,149,2,3
...,...,...,...,...,...,...,...,...,...,...
393,Will Grier,2019,0.0,228.0,0.0,4,7,22,1,0
394,Will Levis,2023,115.0,1808.0,8.0,4,25,57,4,1
395,Zach Wilson,2021,163.0,2334.0,9.0,11,29,185,1,4
396,Zach Wilson,2022,105.0,1688.0,6.0,7,28,102,1,1


In [40]:
name_fantasy_dict = {}

# Iterate over the rows of the DataFrame
for index, row in QB2.iterrows():
    name = row['Name']
    year = row['Year']
    fantasy_points = row['Fantasy Points']
    
    if name not in name_fantasy_dict:  # If the player is not already in the dictionary
        name_fantasy_dict[name] = [fantasy_points if y == year else 0 for y in range(2019, 2024)]
    else:
        # If the player is already in the dictionary, update the corresponding value list
        name_fantasy_dict[name][year - 2019] = fantasy_points

# Print the resulting dictionary
print(name_fantasy_dict)

{'A.J. McCarron': [16.0, 0.0, 0, 0, 0.0], 'Aaron Rodgers': [322.0, 469.0, 394.0, 286.0, 0.0], "Aidan O'Connell": [0, 0, 0, 0, 144.0], 'Alex Smith': [0, 79.0, 0, 0, 0], 'Alex Tanney': [0.0, 0, 0, 0, 0], 'Andy Dalton': [232.0, 158.0, 96.0, 204.0, 27.0], 'Anthony Brown': [0, 0, 0, 6.0, 0], 'Anthony Richardson': [0, 0, 0, 0, 76.0], 'Bailey Zappe': [0, 0, 0, 51.0, 76.0], 'Baker Mayfield': [262.0, 288.0, 205.0, 134.0, 319.0], 'Ben DiNucci': [0, 8.0, 0, 0, 0], 'Ben Roethlisberger': [12.0, 326.0, 258.0, 0, 0], 'Blaine Gabbert': [0, 18.0, 1.0, 7.0, 5.0], 'Blake Bortles': [0.0, 0, 0, 0, 0], 'Brandon Allen': [35.0, 56.0, 17.0, 0.0, 0], 'Brett Hundley': [4.0, 0, 0, 0, 0], 'Brett Rypien': [0, 15.0, 0.0, 20.0, 4.0], 'Brian Hoyer': [29.0, 2.0, 14.0, 1.0, 5.0], 'Brock Purdy': [0, 0, 0, 128.0, 347.0], 'Bryce Perkins': [0, 0, 0, 15.0, 0], 'Bryce Young': [0, 0, 0, 0, 170.0], 'C.J. Beathard': [0, 64.0, 1.0, -1.0, 20.0], 'C.J. Stroud': [0, 0, 0, 0, 314.0], 'Cam Newton': [18.0, 258.0, 90.0, 0, 0], 'Carson W

In [41]:
for index, row in player_predictions.iterrows():
    player = row['Player']
    predicted_points = row['Predicted_Fantasy_Points_2024']
    if player in name_fantasy_dict:
        name_fantasy_dict[player].append(predicted_points)
   

print(name_fantasy_dict)

{'A.J. McCarron': [16.0, 0.0, 0, 0, 0.0, 11.74], 'Aaron Rodgers': [322.0, 469.0, 394.0, 286.0, 0.0, 277.53], "Aidan O'Connell": [0, 0, 0, 0, 144.0, 150.91], 'Alex Smith': [0, 79.0, 0, 0, 0, 81.32], 'Alex Tanney': [0.0, 0, 0, 0, 0, 11.74], 'Andy Dalton': [232.0, 158.0, 96.0, 204.0, 27.0, 53.49], 'Anthony Brown': [0, 0, 0, 6.0, 0, 11.74], 'Anthony Richardson': [0, 0, 0, 0, 76.0, 46.53], 'Bailey Zappe': [0, 0, 0, 51.0, 76.0, 81.32], 'Baker Mayfield': [262.0, 288.0, 205.0, 134.0, 319.0, 240.58], 'Ben DiNucci': [0, 8.0, 0, 0, 0, 11.74], 'Ben Roethlisberger': [12.0, 326.0, 258.0, 0, 0, 268.58], 'Blaine Gabbert': [0, 18.0, 1.0, 7.0, 5.0, 11.85], 'Blake Bortles': [0.0, 0, 0, 0, 0, 11.74], 'Brandon Allen': [35.0, 56.0, 17.0, 0.0, 0, 38.2], 'Brett Hundley': [4.0, 0, 0, 0, 0, 11.74], 'Brett Rypien': [0, 15.0, 0.0, 20.0, 4.0, 11.74], 'Brian Hoyer': [29.0, 2.0, 14.0, 1.0, 5.0, 20.46], 'Brock Purdy': [0, 0, 0, 128.0, 347.0, 371.27], 'Bryce Perkins': [0, 0, 0, 15.0, 0, 23.34], 'Bryce Young': [0, 0, 0

In [42]:
import json

In [43]:
json_data = json.dumps(name_fantasy_dict, indent=4)

In [44]:
with open('../Flask/static/QBYearlyPoints.json','w') as json_file:
    json_file.write(json_data)

In [ ]:
####FINAL MODEL WE USED #####

In [ ]:
X = QB.drop(['Fantasy Points'], axis=1)
y = QB['Fantasy Points']


numeric_features = ['PassingTDs']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
 
    ]
)
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

X_train_subset = X_train[X_train.index.year < 2023]
y_train_subset = y_train[y_train.index.year < 2023]
pipeline.fit(X_train_subset, y_train_subset)

X_test_subset = X_test[X_test.index.year == 2023]
y_test_subset = y_test[y_test.index.year == 2023]
y_pred = pipeline.predict(X_test_subset)

mse = mean_squared_error(y_test_subset, y_pred)
r2 = r2_score(y_test_subset, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")